# Training and using a NER model

This notebook shows how to train a new toponym recognition (NER) model.

We start by importing some libraries, and the `recogniser` script from the `geoparser` folder:

In [ ]:
import os
import sys

sys.path.insert(0, os.path.abspath(os.path.pardir))
from geoparser import recogniser

Create a `myner` object of the `Recogniser` class.

> **Note:** The train and test sets for training the NER modules are two json files, one for training and one for testing, in which each item/line in the json corresponds to a sentence. Each sentence-dictionary has three key-value pairs (see two examples below): `id` is an ID of the sentence (a string), `tokens` is the list of tokens into which the sentence has been split, and `ner_tags` is the list of annotations per token (in BIO format). The length of `tokens` and `ner_tags` should always be the same.
> ```json
> {"id":"3896239_29","ner_tags":["O","B-STREET","I-STREET","O","O","O","B-BUILDING","I-BUILDING","O","O","O","O","O","O","O","O","O","O"],"tokens":[",","Old","Millgate",",","to","the","Collegiate","Church",",","where","they","arrived","a","little","after","ten","oclock","."]}
> {"id":"8262498_11","ner_tags":["O","O","O","O","O","O","O","O","O","O","O","B-LOC","O","B-LOC","O","O","O","O","O","O"],"tokens":["On","the","\u2018","JSth","November","the","ship","Santo","Christo",",","from","Monteveido","to","Cadiz",",","with","hides","and","copper","."]}
> ```
>
> **Note:** If your tagset is different from the one we use (`["O", "B-LOC", "I-LOC", "B-STREET", "I-STREET", "B-BUILDING", "I-BUILDING", "B-OTHER", "I-OTHER", "B-FICTION", "I-FICTION"]`), you will need to manually modify the mapping dictionaries `map_tag_label()` and `encode_dict()` in the script [utils/ner.py](https://github.com/Living-with-machines/toponym-resolution/blob/main/utils/ner.py) accordingly. In the future, we plan to improve this bit so it's automatically done, but for the time being you will need to do this manually.

In [ ]:
myner = recogniser.Recogniser(
    model_name="your_chosen_model_name",  # The name of your NER model (note: we may append suffixes to it).
    model=None,  # We'll store the NER model here, leave this empty.
    pipe=None,  # We'll store the NER pipeline here, leave this empty.
    base_model="khosseini/bert_1760_1900",  # Base model to fine-tune for NER. The value can be: either 
                                            # your local path to a model or the huggingface path.
                                            # In this case, we use the huggingface path:
                                            # https://huggingface.co/khosseini/bert_1760_1900). You can
                                            # chose any other model from the HuggingFace hub, as long as it's
                                            # trained on the "Fill-Mask" objective (filter by task).
    train_dataset="../experiments/outputs/data/lwm/ner_df_train.json",  # Path to the json file containing the training set (see note above).
    test_dataset="../experiments/outputs/data/lwm/ner_df_dev.json",  # Path to the json file containing the test set (see note above).
    output_model_path="../resources/models/",  # Path where the NER model will be stored.
    training_args={
        "learning_rate": 5e-5,
        "batch_size": 16,
        "num_train_epochs": 4,
        "weight_decay": 0.01,
    }, # Training arguments: you can change them.
    overwrite_training=True,  # Set to True if you want to overwrite an existing model with the same name.
    do_test=True,  # Set to True if you want to perform the training on test mode (the string "_test" will be appended to your model name).
    training_tagset="fine",  # Options are: "coarse" (will cluster all tags under LOC) or "fine" (will keep the different tags). See note above.
)

Now train the model:

In [ ]:
myner.train()

Now, to use the model you have just trained, you'll need to load a NER pipeline:

In [ ]:
myner.model, myner.pipe = myner.create_pipeline()

And, finally, use the newly trained model to predict the named entities in a sentence.

In [ ]:
sentence = "A remarkable case of rattening has just occurred in the building trade at Sheffield."

predictions = myner.ner_predict(sentence)
print(predictions) # Note that, if you've trained the model in the test mode, the model will probably not identify "Sheffield" as a location.